<span style="color:red">WIPO PUBLIC</span>

# Mining search
*version 5.0 -- AD 2019-07-24*

This notebook performs a search of the WIPOstats data server for Mining patents based on the technology search performed for WIPO Economics Reserach Working Paper no. 56 [Measuring Innovation in the Mining Industry with Patents](https://www.wipo.int/publications/en/details.jsp?id=4420&plang=EN).

## Project Specific files

This notebook reads the following files for the search in the server:

**search_files/..**

* KWsearch.csv
* mining_categories.csv
* tech_search.csv

In [1]:
#check files
with open('search_files/KWsearch.csv') as file:
    print(file)
with open('search_files/mining_categories.csv') as file:
    print(file)
with open('search_files/tech_search.csv') as file:
    print(file)

<_io.TextIOWrapper name='search_files/KWsearch.csv' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='search_files/mining_categories.csv' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='search_files/tech_search.csv' mode='r' encoding='cp1252'>


## Global Variables

In [2]:
database_name = "patstat_plus"
print(database_name)

patstat_plus


### Tables created in the server

In [3]:
kw_table_name = database_name+".mining2019_kw"
print(kw_table_name)
e21_table_name = database_name+".mining2019_e21"
print(e21_table_name)
search_table_name = database_name+".mining2019_patents"
print(search_table_name)
min_table_name = database_name+".mining2019_min_patents"
print(min_table_name)
mining_families = database_name+".mining2019_first_fam"
print(mining_families)
#mining_families_inpadoc = database_name+".mining2019_inpadoc"
#print(mining_families_inpadoc)
#mining_families_docdb = database_name+".mining2019_docdb"
#print(mining_families_docdb)

patstat_plus.mining2019_kw
patstat_plus.mining2019_e21
patstat_plus.mining2019_patents
patstat_plus.mining2019_min_patents
patstat_plus.mining2019_first_fam


##### Import packages

In [4]:
import numpy as np
import pandas as pd
import pyodbc
import time
import os
import csv

##### Create conntection to WIPOstats server

In [5]:
# global variables: database_name
# This connect works for the WIPO MySQL database, other databases will need alternative database connections
# UID user name deleted for sharing
# PWD password deleted for sharing
con = pyodbc.connect("""DRIVER={MySQL ODBC 8.0 Unicode Driver}; 
                        SERVER=statsql.wipo.int; 
                        DATABASE="""+database_name+"""; 
                        UID = ;
                        PWD = """)

cur = con.cursor()

#### Keyword Search
This is a search for mining patents based on keywords. We create a subset of patents that contain keywords related to mining in the title or abstract, based on the keywords listed in Appendix D of the [working paper](https://www.wipo.int/publications/en/details.jsp?id=4420&plang=EN). 

Six categories of keywords were used in five languages (English, German, Spanish, French and Portuguese) which are the most frequently occurring languages in the abstract and title tables of PATSTAT. The main keywords are only searched in fields in that language, i.e. English keywords were only searched in English titles or abstracts. Ore terms are searched in all abstracts regardless of language. The keywords were searched in the titles and abstracts and assigned a general keyword term, listed in Appendix D.

Some keyword exclusions are noted, such as *mining NOT data mining*. 

In [7]:
# global variable kw_table_name
# local variables:
KWfile = 'search_files/KWsearch.csv'

# create table
SQL = """DROP TABLE if exists """+kw_table_name
cur.execute(SQL)
con.commit()
SQL = """CREATE TABLE """+kw_table_name+""" (appln_id INT
                                              ,index(appln_id)
                                              ,lang CHAR(2)
                                              ,keywords CHAR(20)
                                              ,source CHAR(5))"""
cur.execute(SQL)
con.commit()

# perform search
with open(KWfile) as file:
    filereader = csv.reader(file, dialect='excel')
    next(filereader)
    for line in filereader:
        if line[0] == 'kw1':
            SQL_t = """INSERT INTO """+kw_table_name+"""
                        SELECT DISTINCT t2.appln_id, t2.appln_title_lg, '"""+line[1]+"""' as keywords, 'title' as source
                        FROM patstat.tls202_appln_title t2 
                        WHERE t2.appln_title rlike '"""+line[3]+"""'
                        AND t2.appln_title_lg = '"""+line[2]+"""'"""
            SQL_a = """INSERT INTO """+kw_table_name+"""
                        SELECT DISTINCT t3.appln_id, t3.appln_abstract_lg, '"""+line[1]+"""' as keywords, 'abstr' as source
                        FROM patstat.tls203_appln_abstr t3 
                        WHERE t3.appln_abstract rlike '"""+line[3]+"""'
                        AND t3.appln_abstract_lg = '"""+line[2]+"""'"""
            cur.execute(SQL_t)
            con.commit()
            cur.execute(SQL_a)
            con.commit()
        elif line[0] == 'kw3':
            SQL_t = """INSERT INTO """+kw_table_name+"""
                        SELECT DISTINCT t2.appln_id, t2.appln_title_lg, '"""+line[1]+"""' as keywords, 'title' as source
                        FROM patstat.tls202_appln_title t2 
                        WHERE t2.appln_title rlike '"""+line[3]+"""'
                        AND t2.appln_title not rlike '"""+line[4]+"""'
                        AND t2.appln_title_lg = '"""+line[2]+"""'"""
            SQL_a = """INSERT INTO """+kw_table_name+"""
                        SELECT DISTINCT t3.appln_id, t3.appln_abstract_lg, '"""+line[1]+"""' as keywords, 'abstr' as source
                        FROM patstat.tls203_appln_abstr t3 
                        WHERE t3.appln_abstract rlike '"""+line[3]+"""'
                        AND t3.appln_abstract not rlike '"""+line[4]+"""'
                        AND t3.appln_abstract_lg = '"""+line[2]+"""'"""
            cur.execute(SQL_t)
            con.commit()
            cur.execute(SQL_a)
            con.commit()            
        elif line[0] == 'kw2':
            SQL_t = """INSERT INTO """+kw_table_name+"""
                        SELECT DISTINCT t2.appln_id, t2.appln_title_lg, '"""+line[1]+"""' as keywords, 'title' as source
                        FROM patstat.tls202_appln_title t2 
                        WHERE t2.appln_title rlike '"""+line[3]+"""'"""
            SQL_a = """INSERT INTO """+kw_table_name+"""
                        SELECT DISTINCT t3.appln_id, t3.appln_abstract_lg, '"""+line[1]+"""' as keywords, 'abstr' as source
                        FROM patstat.tls203_appln_abstr t3 
                        WHERE t3.appln_abstract rlike '"""+line[3]+"""'"""
            cur.execute(SQL_t)
            con.commit()
            cur.execute(SQL_a)
            con.commit()
        else: end

#### E21 search
A second subset of data of all patents comprising at least one [E21](https://www.wipo.int/classifications/ipc/ipcpub/?notion=scheme&version=20190101&symbol=E21) IPC or CPC mark.

In [8]:
# global variables: e21_table_name

# create table of E21 patents
SQL = """DROP TABLE if exists """+e21_table_name
cur.execute(SQL)
con.commit()
SQL = """CREATE TABLE """+e21_table_name+""" (index(appln_id))
SELECT DISTINCT appln_id
FROM patstat.tls209_appln_ipc t9
WHERE t9.ipc_class_symbol like 'E21%'"""
cur.execute(SQL)
con.commit()
SQL = """INSERT INTO """+e21_table_name+"""
SELECT DISTINCT appln_id
FROM patstat.tls224_appln_cpc t24 
WHERE t24.cpc_class_symbol like 'E21%'"""
cur.execute(SQL)
con.commit()

#### Preliminary mining patents
These two subsets of data were used as a subset of the whole PATSTAT database to search within, schematically represented below. At the same time each search string is assigned to a mining category. The mining category id numbers are listed in Appendix F of the [working paper](https://www.wipo.int/publications/en/details.jsp?id=4420&plang=EN).

<img src="Search schema.jpg" alt="Search schema" width="600">
 

In [ ]:
# global variables: kw_table_name, e21_table_name, search_table_name
# local variables:
searchfile = 'search_files/tech_search.csv'

# create table
SQL = """DROP TABLE if exists """+search_table_name
cur.execute(SQL)
con.commit()
SQL = """CREATE TABLE """+search_table_name+""" 
            (appln_id INT
            ,index(appln_id)
            ,mining_cat_id DECIMAL(5,2)
            ,index(mining_cat_id)
            ,source CHAR(20))"""
cur.execute(SQL)
con.commit()

# Search/Sort mining patents based on KW, IPC and combinations
 
with open(searchfile) as file:
    filereader = csv.reader(file, dialect='excel')
    next(filereader)
    for line in filereader:
        if line[0] == 'allKW':
            SQL_i = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t9.appln_id, """+line[2]+""" as mining_cat_id, 'IPC + all kw' as source
                        FROM patstat.tls209_appln_ipc t9
                        INNER JOIN """+kw_table_name+""" mkw using(appln_id)
                        WHERE t9.ipc_class_symbol like '"""+line[3]+"""%'"""
            cur.execute(SQL_i)
            con.commit()
            SQL_c = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t24.appln_id, """+line[2]+""" as mining_cat_id, 'CPC + all kw' as source
                        FROM patstat.tls224_appln_cpc t24
                        INNER JOIN """+kw_table_name+""" mkw using(appln_id)
                        WHERE t24.cpc_class_symbol like '"""+line[3]+"""%'"""
            cur.execute(SQL_c)
            con.commit()

        elif line[0] == 'IPCandE21':
            SQL_i = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t9.appln_id, """+line[2]+""" as mining_cat_id, 'IPC combo' as source
                        FROM patstat.tls209_appln_ipc t9
                        INNER JOIN """+e21_table_name+""" e21 using(appln_id)
                        WHERE t9.ipc_class_symbol like '"""+line[3]+"""%'"""
            cur.execute(SQL_i)
            con.commit()
            SQL_c = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t24.appln_id, """+line[2]+""" as mining_cat_id, 'CPC combo' as source
                        FROM patstat.tls224_appln_cpc t24
                        INNER JOIN """+e21_table_name+""" e21 using(appln_id)
                        WHERE t24.cpc_class_symbol like '"""+line[3]+"""%'"""
            cur.execute(SQL_c)
            con.commit()
            
        elif line[0] == 'selectKW':
            SQL_i = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t9.appln_id, """+line[2]+""" as mining_cat_id, 'IPC + sel kw' as source
                        FROM patstat.tls209_appln_ipc t9
                        INNER JOIN """+kw_table_name+""" mkw using(appln_id)
                        WHERE t9.ipc_class_symbol like '"""+line[3]+"""%'
                        AND ("""+line[4]+""") """
            cur.execute(SQL_i)
            con.commit()
            SQL_c = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t24.appln_id, """+line[2]+""" as mining_cat_id, 'CPC + sel kw' as source
                        FROM patstat.tls224_appln_cpc t24
                        INNER JOIN """+kw_table_name+""" mkw using(appln_id)
                        WHERE t24.cpc_class_symbol like '"""+line[3]+"""%'
                        AND ("""+line[4]+""") """
            cur.execute(SQL_c)
            con.commit()

        elif line[0] == 'E21notIPC':
            SQL_i = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t9.appln_id, """+line[2]+""" as mining_cat_id, 'IPC combo NOT' as source
                        FROM patstat.tls209_appln_ipc t9
                        INNER JOIN """+e21_table_name+""" e21 on t9.appln_id = e21.appln_id
                        INNER JOIN patstat.tls209_appln_ipc t9x on t9.appln_id = t9x.appln_id
                        WHERE t9.ipc_class_symbol like '"""+line[3]+"""%'
                        AND t9x.ipc_class_symbol not like '"""+line[5]+"""%'"""
            cur.execute(SQL_i)
            con.commit()
            SQL_c = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t24.appln_id, """+line[2]+""" as mining_cat_id, 'CPC combo NOT' as source
                        FROM patstat.tls224_appln_cpc t24
                        INNER JOIN """+e21_table_name+""" e21 on t24.appln_id = e21.appln_id
                        INNER JOIN patstat.tls224_appln_cpc t24x on t24.appln_id = t24x.appln_id
                        WHERE t24.cpc_class_symbol like '"""+line[3]+"""%'
                        AND t24x.cpc_class_symbol not like '"""+line[5]+"""%'"""
            cur.execute(SQL_c)
            con.commit()            

        elif line[0] == 'IPConly':
            SQL_i = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t9.appln_id, """+line[2]+""" as mining_cat_id, 'IPC' as source
                        FROM patstat.tls209_appln_ipc t9
                        WHERE t9.ipc_class_symbol like '"""+line[3]+"""%'"""
            cur.execute(SQL_i)
            con.commit()
            SQL_c = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t24.appln_id, """+line[2]+""" as mining_cat_id, 'CPC' as source
                        FROM patstat.tls224_appln_cpc t24
                        WHERE t24.cpc_class_symbol like '"""+line[3]+"""%'"""
            cur.execute(SQL_c)
            con.commit()

        elif line[0] == 'KWnotIPC':
            SQL_i = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t9.appln_id, """+line[2]+""" as mining_cat_id, 'IPC + sel kw NOT' as source
                        FROM patstat.tls209_appln_ipc t9
                        INNER JOIN """+kw_table_name+""" mkw using(appln_id)
                        INNER JOIN patstat.tls209_appln_ipc t9x on t9.appln_id = t9x.appln_id
                        WHERE ("""+line[4]+""") 
                        AND t9.ipc_class_symbol like '"""+line[3]+"""%'
                        AND t9x.ipc_class_symbol not like '"""+line[5]+"""%'"""
            cur.execute(SQL_i)
            con.commit()
            SQL_c = """INSERT INTO """+search_table_name+"""
                        SELECT DISTINCT t24.appln_id, """+line[2]+""" as mining_cat_id, 'CPC + sel kw NOT' as source
                        FROM patstat.tls224_appln_cpc t24
                        INNER JOIN """+kw_table_name+""" mkw using(appln_id)
                        INNER JOIN patstat.tls224_appln_cpc t24x on t24.appln_id = t24x.appln_id
                        WHERE ("""+line[4]+""") 
                        AND t24.cpc_class_symbol like '"""+line[3]+"""%'
                        AND t24x.cpc_class_symbol not like '"""+line[5]+"""%'"""
            cur.execute(SQL_c)
            con.commit()

        else: end

The residual keyword results are also assigned a category.

In [ ]:
# global variables: search_table_name, kw_table_name
# local variables:

# 9.97, mining
SQL_997 = """INSERT INTO """+search_table_name+"""
                SELECT DISTINCT appln_id, 9.97 as mining_cat_id, source as source
                FROM """+kw_table_name+""" mkw
                WHERE (mkw.keywords = 'mining')"""
# 9.98, mine
SQL_998 = """INSERT INTO """+search_table_name+"""
                SELECT DISTINCT appln_id, 9.98 as mining_cat_id, source as source
                FROM """+kw_table_name+""" mkw
                WHERE (mkw.keywords = 'mine')"""
# 109.1, ore
SQL_1091 = """INSERT INTO """+search_table_name+"""
                SELECT DISTINCT appln_id, 109.1 as mining_cat_id, source as source
                FROM """+kw_table_name+""" mkw
                WHERE (mkw.keywords = 'ore')"""
# 109.2, ore%
SQL_1092 = """INSERT INTO """+search_table_name+"""
                SELECT DISTINCT appln_id, 109.2 as mining_cat_id, source as source
                FROM """+kw_table_name+""" mkw
                WHERE (mkw.keywords like 'ore:%')"""
# 109.3, mineral
SQL_1093 = """INSERT INTO """+search_table_name+"""
                SELECT DISTINCT appln_id, 109.3 as mining_cat_id, source as source
                FROM """+kw_table_name+""" mkw
                WHERE (mkw.keywords = 'mineral')"""
# 109.4, coal
SQL_1094 = """INSERT INTO """+search_table_name+"""
                SELECT DISTINCT appln_id, 109.4 as mining_cat_id, source as source
                FROM """+kw_table_name+""" mkw
                WHERE (mkw.keywords = 'coal')"""

cur.execute(SQL_997)
con.commit()
cur.execute(SQL_998)
con.commit()
cur.execute(SQL_1091)
con.commit()
cur.execute(SQL_1092)
con.commit()
cur.execute(SQL_1093)
con.commit()
cur.execute(SQL_1094)
con.commit()

Some minor renumbering, dropping observations in (early) exclusions that include E21


In [ ]:
# global variables: search_table_name, e21_table_name
# local variables:

SQL = """DELETE FROM """+search_table_name+"""
            WHERE mining_cat_id < 1
            AND appln_id in (SELECT appln_id
                            FROM """+e21_table_name+""" e21)"""
cur.execute(SQL)
con.commit()

### Cateogry minimization

In order to assign one category to each patent (based on a manually assigned hierarchy of technologies that occur in the mining sector), each category was assigned a mining category id number when performing the search above. Now we will take the minimum of all the mining category id numbers assigned to each patent to determine the principal category for each patent. 

For example, if a patent application is assigned both categories 3.3 (separation) and 6.1 (exploration), then the minimum of 3.3 would be the final category for that patent application.

In [ ]:
# global variables: search_table_name, min_table_name
# local variables:

# create table
SQL = """DROP TABLE if exists """+min_table_name
cur.execute(SQL)
con.commit()
SQL = """CREATE TABLE """+min_table_name+""" AS
            SELECT appln_id, min(mining_cat_id) as min_mining_cat_id
            FROM """+search_table_name+"""
            GROUP BY appln_id"""
cur.execute(SQL)
con.commit()
SQL = """ALTER TABLE """+min_table_name+""" ADD index(appln_id,min_mining_cat_id)"""
cur.execute(SQL)
con.commit()

Renumber in process Exclusions

In [ ]:
# global variables: min_table_name

SQL_311 = """UPDATE """+min_table_name+""" mt
                SET mt.min_mining_cat_id = 100.91
                WHERE mt.min_mining_cat_id = 3.11"""
SQL_441 = """UPDATE """+min_table_name+""" mt
                SET mt.min_mining_cat_id = 100.92
                WHERE mt.min_mining_cat_id = 4.41"""
SQL_471 = """UPDATE """+min_table_name+""" mt
                SET mt.min_mining_cat_id = 100.93
                WHERE mt.min_mining_cat_id = 4.71"""
SQL_961 = """UPDATE """+min_table_name+""" mt
                SET mt.min_mining_cat_id = 100.94
                WHERE mt.min_mining_cat_id = 9.61"""
SQL_631 = """UPDATE """+min_table_name+""" mt
                SET mt.min_mining_cat_id = 100.94
                WHERE mt.min_mining_cat_id = 6.31"""
cur.execute(SQL_311)
con.commit()
cur.execute(SQL_441)
con.commit()
cur.execute(SQL_471)
con.commit()
cur.execute(SQL_961)
con.commit()
cur.execute(SQL_631)
con.commit()


### Create Mining Families Table
Using WIPO patent famlies.

*N.B. if other families are prefered, join at this point to patstat.tls201_appln to retrive inpadoc or docdb families using the second cell*

In [ ]:
# create families table in server
SQL = """DROP TABLE if exists """+mining_families
cur.execute(SQL)
con.commit()
SQL = """CREATE TABLE """+mining_families+""" (index(appln_id,fam_id))
            SELECT DISTINCT ffr2.fam_id, ffr2.appln_id, min(w.min_mining_cat_id) as min_mining_cat_id 
            FROM """+min_table_name+""" w
            INNER JOIN gstat_miscellaneous.family_appln_relation ffr on w.appln_id = ffr.appln_id
            INNER JOIN gstat_miscellaneous.family_appln_relation ffr2 on ffr.fam_id = ffr2.fam_id
            GROUP BY ffr2.fam_id, ffr2.appln_id"""
cur.execute(SQL)
con.commit()

In [ ]:
# # create alternative families table in server
# # INPADOC
# SQL = """DROP TABLE if exists """+mining_families_inpadoc
# cur.execute(SQL)
# con.commit()
# SQL = """CREATE TABLE """+mining_families_inpadoc+""" (index(appln_id,inpadoc_family_id))
#             SELECT DISTINCT t1a.inpadoc_family_id, t1b.appln_id, w.min_mining_cat_id
#             FROM """+min_table_name+""" w
#             INNER JOIN patstat.tls201_appln t1a on w.appln_id=t1a.appln_id
#             INNER JOIN patstat.tls201_appln t1b on t1a.inpadoc_family_id=t1b.inpadoc_family_id"""
# cur.execute(SQL)
# con.commit()

# # DOCDB
# SQL = """DROP TABLE if exists """+mining_families_docdb
# cur.execute(SQL)
# con.commit()
# SQL = """CREATE TABLE """+mining_families_docdb+""" (index(appln_id,docdb_family_id))
#             SELECT DISTINCT t1a.docdb_family_id, t1b.appln_id, w.min_mining_cat_id
#             FROM """+min_table_name+""" w
#             INNER JOIN patstat.tls201_appln t1a on w.appln_id=t1a.appln_id
#             INNER JOIN patstat.tls201_appln t1b on t1a.docdb_family_id=t1b.docdb_family_id"""
# cur.execute(SQL)
# con.commit()

# # add first filing flag - INPADOC
# SQL = """ALTER TABLE """+mining_families_inpadoc+"""
#             ADD COLUMN first_filing tinyint default 1, add index(first_filing)"""
# cur.execute(SQL)
# con.commit()
# SQL = """UPDATE """+mining_families_inpadoc+""" w 
#             INNER JOIN patstat.tls201_appln t1 using(appln_id)
#             SET w.first_filing=0 
#             WHERE w.first_filing=1 and (t1.appln_kind not in ('A','U','W') or t1.internat_appln_id>0)"""
# cur.execute(SQL)
# con.commit()
# SQL = """UPDATE """+mining_families_inpadoc+""" w 
#             INNER JOIN patstat.tls204_appln_prior t4 on w.appln_id=t4.appln_id
#             INNER JOIN patstat.tls201_appln t1 on t4.prior_appln_id=t1.appln_id
#             SET w.first_filing=0 
#             WHERE w.first_filing=1 and t1.appln_kind in ('A','U','W')"""
# cur.execute(SQL)
# con.commit()
# SQL = """UPDATE """+mining_families_inpadoc+""" w 
#             INNER JOIN patstat.tls216_appln_contn t16 on w.appln_id=t16.appln_id
#             INNER JOIN patstat.tls201_appln t1 on t16.parent_appln_id=t1.appln_id
#             SET w.first_filing=0 
#             WHERE w.first_filing=1 and t1.appln_kind in ('A','U','W')"""
# cur.execute(SQL)
# con.commit()

# # add first filing flag - DOCDB
# SQL = """ALTER TABLE """+mining_families_docdb+"""
#             ADD COLUMN first_filing tinyint default 1, add index(first_filing)"""
# cur.execute(SQL)
# con.commit()
# SQL = """UPDATE """+mining_families_docdb+""" w 
#             INNER JOIN patstat.tls201_appln t1 using(appln_id)
#             SET w.first_filing=0 
#             WHERE w.first_filing=1 and (t1.appln_kind not in ('A','U','W') or t1.internat_appln_id>0)"""
# cur.execute(SQL)
# con.commit()
# SQL = """UPDATE """+mining_families_docdb+""" w 
#             INNER JOIN patstat.tls204_appln_prior t4 on w.appln_id=t4.appln_id
#             INNER JOIN patstat.tls201_appln t1 on t4.prior_appln_id=t1.appln_id
#             SET w.first_filing=0 
#             WHERE w.first_filing=1 and t1.appln_kind in ('A','U','W')"""
# cur.execute(SQL)
# con.commit()
# SQL = """UPDATE """+mining_families_docdb+""" w 
#             INNER JOIN patstat.tls216_appln_contn t16 on w.appln_id=t16.appln_id
#             INNER JOIN patstat.tls201_appln t1 on t16.parent_appln_id=t1.appln_id
#             SET w.first_filing=0 
#             WHERE w.first_filing=1 and t1.appln_kind in ('A','U','W')"""
# cur.execute(SQL)
# con.commit()

##### Close connection to server

In [ ]:
con.close()